In [2]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re

# Read the file content
def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r" structure (?P<structure>.+)\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_walltime\s+(?P<gpu_walltime>[\d.]+)"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
      'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
      'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
      'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_walltime'
  ]
  # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'v100'
!rm -rf notes/capture/{option}
!mkdir -p notes/capture/{option}
!rm -rf notes/data/{option}
!mkdir -p notes/data/{option}
for ncore in tqdm([1, 2, 4, 8, 16, 32]):
  ncore = str(ncore)
  !OMP_NUM_THREADS={ncore} meson test -C build/ hamiltonian --verbose -t 0 > notes/capture/{option}/{ncore}cpu.txt
  df = parse_to_datafame(f"notes/capture/{option}/{ncore}cpu.txt")
  df.to_csv(f"notes/data/{option}/{ncore}cpu.csv", index=False)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [08:58<00:00, 89.81s/it] 


In [3]:
!nproc
!lscpu

32
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   32
  On-line CPU(s) list:    0-31
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   1
    Core(s) per socket:   16
    Socket(s):            2
    Stepping:             4
    CPU max MHz:          3700.0000
    CPU min MHz:          1000.0000
    BogoMIPS:             4200.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc cpuid aperfmperf pni pc

In [4]:
!nvidia-smi

Wed May 21 08:02:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:18:00.0 Off |                    0 |
| N/A   42C    P0             67W /  300W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----